In [1]:
import torch 
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import pandas as pd
import timeit
import numpy as np


c:\Users\avrey\anaconda3\envs\iml.env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np


def spherical_to_cartesian(r, theta, phi,normalize=True):
    out = np.array(
        [
            r * np.sin(np.deg2rad(theta)) * np.cos(np.deg2rad(phi)),
            r * np.sin(np.deg2rad(theta)) * np.sin(np.deg2rad(phi)),
            r * np.cos(np.deg2rad(theta)),
        ]
    )
    if normalize:
        return out / np.linalg.norm(out)
    return out
def cartesian_to_spherical(x, y, z):
    r = np.linalg.norm([x, y, z])
    theta = np.rad2deg(np.arccos(z / r))
    phi = np.rad2deg(np.arctan2(y, x))
    return r, theta, phi

In [26]:
def cone_by_spherical(
    x: float, y: float, z: float, theta: int, phi: int, r: float = 1
) -> go.Cone:
    u, v, w = spherical_to_cartesian(r, theta, phi,normalize=False)
    return go.Cone(x=[x], y=[y], z=[z], u=[u], v=[v], w=[w],anchor="tail",sizemode="absolute")

In [29]:
go.Figure(data=cone_by_spherical(0, 0, 0, 180, 270)).update_layout(
        scene=dict(
            aspectratio=dict(x=1, y=1, z=0.8),
            camera_eye=dict(x=1.2, y=1.2, z=0.6),
            xaxis=dict(
                nticks=4,
                range=[-2, 2],
            ),
            yaxis=dict(
                nticks=4,
                range=[-2, 2],
            ),
            zaxis=dict(
                nticks=4,
                range=[-2, 2],
            ),
        )).show()

In [ ]:
import random
import plotly.graph_objects as go

# Define the parameters of the simulation
num_birds = 50
max_speed = 5
max_force = 0.1
neighborhood_radius = 50
alignment_weight = 1
cohesion_weight = 1
separation_weight = 1

# Define the initial positions and velocities of the birds
positions = [(random.uniform(-100, 100), random.uniform(-100, 100)) for _ in range(num_birds)]
velocities = [(random.uniform(-1, 1), random.uniform(-1, 1)) for _ in range(num_birds)]

# Define the update function for the birds
def update_birds(positions, velocities):
    new_positions = []
    new_velocities = []
    for i, (x, y) in enumerate(positions):
        # Compute the alignment, cohesion, and separation vectors
        alignment = (0, 0)
        cohesion = (0, 0)
        separation = (0, 0)
        num_neighbors = 0
        for j, (x2, y2) in enumerate(positions):
            if i != j and (x2 - x) ** 2 + (y2 - y) ** 2 < neighborhood_radius ** 2:
                alignment = (alignment[0] + velocities[j][0], alignment[1] + velocities[j][1])
                cohesion = (cohesion[0] + x2, cohesion[1] + y2)
                separation = (separation[0] + (x - x2), separation[1] + (y - y2))
                num_neighbors += 1
        if num_neighbors > 0:
            alignment = (alignment[0] / num_neighbors, alignment[1] / num_neighbors)
            alignment_mag = (alignment[0] ** 2 + alignment[1] ** 2) ** 0.5
            if alignment_mag > 0:
                alignment = (alignment[0] / alignment_mag * max_speed, alignment[1] / alignment_mag * max_speed)
            cohesion = (cohesion[0] / num_neighbors, cohesion[1] / num_neighbors)
            cohesion = ((cohesion[0] - x) / 100 * max_speed, (cohesion[1] - y) / 100 * max_speed)
            separation_mag = (separation[0] ** 2 + separation[1] ** 2) ** 0.5
            if separation_mag > 0:
                separation = (separation[0] / separation_mag * max_speed, separation[1] / separation_mag * max_speed)
        # Compute the new velocity and position of the bird
        velocity = (velocities[i][0] + alignment[0] * alignment_weight + cohesion[0] * cohesion_weight + separation[0] * separation_weight, velocities[i][1] + alignment[1] * alignment_weight + cohesion[1] * cohesion_weight + separation[1] * separation_weight)
        velocity_mag = (velocity[0] ** 2 + velocity[1] ** 2) ** 0.5
        if velocity_mag > max_speed:
            velocity = (velocity[0] / velocity_mag * max_speed, velocity[1] / velocity_mag * max_speed)
        position = (x + velocity[0], y + velocity[1])
        new_positions.append(position)
        new_velocities.append(velocity)
    return new_positions, new_velocities

# Create the plot
fig = go.Figure()

# Add the initial positions of the birds to the plot
fig.add_trace(go.Scatter(x=[x for x, y in positions], y=[y for x, y in positions], mode='markers'))

# Run the simulation and update the plot
for i in range(100):
    positions, velocities = update_birds(positions, velocities)
    fig.add_trace(go.Scatter(x=[x for x, y in positions], y=[y for x, y in positions], mode='markers'))
    
# Show the plot
fig.show()

In [ ]:
import random
import torch
import matplotlib.pyplot as plt

# Define the parameters of the simulation
num_birds = 50
max_speed = 5
max_force = 0.1
neighborhood_radius = 50
alignment_weight = 1
cohesion_weight = 1
separation_weight = 1

# Define the initial positions and velocities of the birds
positions = torch.tensor([(torch.uniform(()-100, 100), random.uniform(-100, 100)) for _ in range(num_birds)], requires_grad=True)
velocities = torch.tensor([(random.uniform(-1, 1), random.uniform(-1, 1)) for _ in range(num_birds)], requires_grad=True)

# Define the update function for the birds
def update_birds(positions, velocities):
    new_positions = []
    new_velocities = []
    for i, (x, y) in enumerate(positions):
        # Compute the alignment, cohesion, and separation vectors
        alignment = torch.tensor([0., 0.])
        cohesion = torch.tensor([0., 0.])
        separation = torch.tensor([0., 0.])
        num_neighbors = 0
        for j, (x2, y2) in enumerate(positions):
            if i != j and (x2 - x) ** 2 + (y2 - y) ** 2 < neighborhood_radius ** 2:
                alignment += velocities[j]
                cohesion += torch.tensor([x2, y2])
                separation += torch.tensor([x - x2, y - y2])
                num_neighbors += 1
        if num_neighbors > 0:
            alignment /= num_neighbors
            alignment_mag = torch.norm(alignment)
            if alignment_mag > 0:
                alignment *= max_speed / alignment_mag
            cohesion /= num_neighbors
            cohesion = (cohesion - torch.tensor([x, y])) / 100 * max_speed
            separation_mag = torch.norm(separation)
            if separation_mag > 0:
                separation *= max_speed / separation_mag
        # Compute the new velocity and position of the bird
        velocity = velocities[i] + alignment * alignment_weight + cohesion * cohesion_weight + separation * separation_weight
        velocity_mag = torch.norm(velocity)
        if velocity_mag > max_speed:
            velocity *= max_speed / velocity_mag
        position = positions[i] + velocity
        new_positions.append(position)
        new_velocities.append(velocity)
    return torch.stack(new_positions), torch.stack(new_velocities)

# Create the plot
fig, ax = plt.subplots()

# Add the initial positions of the birds to the plot
ax.scatter(positions[:, 0].detach().numpy(), positions[:, 1].detach().numpy())

# Run the simulation and update the plot
for i in range(100):
    positions, velocities = update_birds(positions, velocities)
    ax.scatter(positions[:, 0].detach().numpy(), positions[:, 1].detach().numpy())
    
# Show the plot
plt.show()

In [49]:
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/vortex.csv")

In [1]:
f = lambda x: x + 1
frames = []
for _ in range(20):
    theta = np.arctan(df["y"] / df["x"])
    df[["x"]] += np.cos(df["x"])
    frames.append(
        go.Frame(
            data=go.Cone(
                x=df["x"],
                y=df["y"],
                z=df["z"],
                u=df["u"],
                v=df["v"],
                w=df["w"],
                colorscale="Blues",
                sizemode="absolute",
                sizeref=40,
            )
        )
    )

fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        title="Start Title",
        updatemenus=[
            dict(
                type="buttons",
                buttons=[dict(label="Play", method="animate", args=[None])],
            )
        ],
    ),
    frames=frames,
)

fig.update_layout(
    scene=dict(
        aspectratio=dict(x=1, y=1, z=0.8),
        camera_eye=dict(x=1.2, y=1.2, z=0.6),
        xaxis=dict(
            nticks=4,
            range=[-20, 20],
        ),
        yaxis=dict(
            nticks=4,
            range=[-20, 20],
        ),
        zaxis=dict(
            nticks=4,
            range=[-20, 20],
        ),
    ),
)

fig.show()

SyntaxError: invalid syntax (429911876.py, line 4)